# Calibration of GEOtop using Nevergrad and GEOtoPy

In [ ]:
# Default

model_path = None
observations_path = None
parameters_path = 'data/parameters/default.csv'
timeout = 300
spinup = {'years': 1}
skip_months = []
default_parameters = {'FirstLayerWidth': 10, 'MaxDepth': 2000}
algorithm = 'NGO'
budget = 4096
num_workers = 256
scheduler_file = None

In [ ]:
from dateutil.relativedelta import relativedelta
from timeit import default_timer as timer
from time import strftime, gmtime

import pandas as pd
import scrapbook as sb
from dask.distributed import Client

from mhpc_project.utils import date_parser
from mhpc_project.parameters import VarSoilParameters
from mhpc_project.models import VarSoilFullModel
from mhpc_project.calibration import Calibration

In [ ]:
# Glue inputs
sb.glue('model_path', model_path)
sb.glue('observations_path', observations_path)
sb.glue('parameters_path', parameters_path)
sb.glue('timeout', timeout)
sb.glue('spinup', spinup)
sb.glue('skip_months', skip_months)
sb.glue('default_parameters', default_parameters)
sb.glue('algorithm', algorithm)
sb.glue('budget', budget)
sb.glue('num_workers', num_workers)

In [ ]:
parameters = VarSoilParameters(parameters_path, default_parameters)
model = VarSoilFullModel(model_path, store=False, timeout=timeout)
observations = pd.read_csv(observations_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)

if spinup:
    start_date = date_parser(model.settings['InitDateDDMMYYYYhhmm'])
    start_date = start_date + relativedelta(**spinup)
    selection = observations.index.map(lambda x: x >= start_date)
    observations = observations[selection]

if skip_months:
    selection = observations.index.map(lambda x: x.month not in skip_months)
    observations = observations[selection]

calibration = Calibration(model, parameters, observations)

In [ ]:
client = Client(scheduler_file=scheduler_file)

In [ ]:
initial_guess = calibration.parameters.instrumentation
calibration.comparison_plots(initial_guess)

In [ ]:
start = timer()
recommendation = calibration(algorithm, budget, num_workers, client)
elapsed = timer() - start

print("elapsed time:", strftime("%T", gmtime(elapsed)))

In [ ]:
calibration.convergence_plot()

In [ ]:
calibration.comparison_plots(recommendation)

In [ ]:
parameters_table = calibration.parameters_table(recommendation)
parameters_table

In [ ]:
# Outputs
sb.glue('parameters_table', parameters_table, 'pandas')
sb.glue('loss', recommendation.loss)
sb.glue('elapsed_time', elapsed)
